*Kaito Arnoni Lacerda* - *Parcours Ingénieur IA*

# <font color=black size=6> P10 - DÉVELOPPEZ UN CHATBOT POUR RÉSERVER DES VACANCES </font> #


# <font color=black size=5> II - Création et entraînement app LUIS </font> #

###  <font size=4 color="#3498DB "> 1. Créer le service « LUIS Language Understanding » <a name="1"></a> </font>

In [66]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from msrest.authentication import CognitiveServicesCredentials
from dotenv import load_dotenv, find_dotenv

import datetime, json, os, time, sys
import requests, time

import pandas as pd
import numpy as np

from tqdm import tqdm

load_dotenv()

In [98]:
authoring_key = os.getenv('AUTHORING_KEY')

authoring_endpoint = os.getenv('AUTHORING_ENDPOINT')

# Instantiate a LUIS client
client = LUISAuthoringClient(authoring_endpoint, CognitiveServicesCredentials(authoring_key))

In [4]:
# CreateApp

def create_app():
# Create a new LUIS app
    app_name = "LuisP10_app {}".format(datetime.datetime.now())
    app_desc = "Projet_10_MVP_Flight_booking"
    app_version = "1"
    app_locale = "en-us"

    app_id = client.apps.add(dict(name=app_name,
                                  initial_version_id=app_version,
                                  description=app_desc,
                                  culture=app_locale))

    print("LUIS app {}\n crée with ID {}".format(app_name, app_id))
    return app_id, app_version

In [5]:
print("Creating application...")
app_id, app_version = create_app()
print()

Creating application...
LUIS app LuisP10_app 2022-04-27 17:31:50.170740
 C crée with ID 4151fd40-18a7-4701-8d67-bf7ec96a511a



###  <font size=4 color="#3498DB "> 2. Création et ajout d'entités <a name="1"></a> </font>


* from_city - ville de départ
* to_city - ville de destination
* from_dt - date aller souhaitée du vo
* to_dt - date retour souhaitée du vol
* budget - budget maximum pour le prix total des billets.   

In [7]:
# addEntities
def add_entities(app_id, app_version):
    from_cityEntityId = client.model.add_entity(app_id, app_version, name="from_city")
    print("from_cityEntityId {} entité crée.".format(from_cityEntityId))
    
    to_cityEntityId = client.model.add_entity(app_id, app_version, name="to_city")
    print("to_cityEntityId {} entité crée.".format(to_cityEntityId))
    
    from_dtEntityId = client.model.add_entity(app_id, app_version, name="from_dt")
    print("from_dtEntityId {} entité crée.".format(from_dtEntityId))
    
    to_dtEntityId = client.model.add_entity(app_id, app_version, name="to_dt")
    print("to_dtEntityId {} entité crée.".format(to_dtEntityId))
    
    budgetEntityId = client.model.add_entity(app_id, app_version, name="budget")
    print("budgetEntityId {} entité crée.".format(budgetEntityId))

In [8]:
print ("Ajout d'entités à l'application...")
add_entities(app_id, app_version)
print ("Opération terminée.")

Ajout d'entités à l'application...
from_cityEntityId 56d4ab5b-6e6a-459c-9212-638eb2b0195d entité crée.
to_cityEntityId 8d33472b-a0bd-45c8-b9ca-ac355fcf6b92 entité crée.
from_dtEntityId a44bdd82-ec5e-468a-b60e-748cfca7d95c entité crée.
to_dtEntityId 6e1e690a-0255-4ac9-a430-fdb3eb35054b entité crée.
budgetEntityId cc8903af-808e-4671-a3cd-2921a3dd3e8a entité crée.
Opération terminée.


###  <font size=4 color="#3498DB "> 3. Création d'une intention pour l’application <a name="1"></a> </font>

In [9]:
#addIntents
def add_intents(app_id, app_version):
    intentId = client.model.add_intent(app_id, app_version, "book_flight")

    print("Intention book_flight {} ajouté.".format(intentId))

In [11]:
print ("Ajout d'intention à l'application...")
add_intents(app_id, app_version)
print ()

Ajout d'intention à l'application...
Intention book_flight e1d0902b-4211-4d0d-9782-b27f2c69d86a ajouté.



###  <font size=4 color="#3498DB "> 4. Ajouter un exemple d’énoncé à une intention <a name="1"></a> </font>

In [31]:
def create_utterance(intent, utterance, *labels):

    text = utterance.lower()

    def label(name, value):
        value = value.lower()
        start = text.index(value)
        return dict(entity_name=name, start_char_index=start,
                    end_char_index=start + len(value))

    return dict(text=text, intent_name=intent,
                entity_labels=[label(n, v) for (n, v) in labels])

def add_utterances(app_id, app_version):
    # Now define the utterances
    utterances = [create_utterance("book_flight", "book flight from London to Paris on May 22, 2021 returning june 21, 2021 for $200",
                                ("from_city", "London"),
                                ("to_city", "Paris"),
                                ("from_dt", "May 22, 2021"),
                                ("to_dt", "june 21, 2021"),
                                ("budget", "$200"))]

    # Add the utterances in batch. You may add any number of example utterances
    # for any number of intents in one call.
    client.examples.batch(app_id, app_version, utterances)
    print("{} Exemple d’énoncé ajouté.".format(len(utterances)))


In [32]:
print ("Ajout d'exemple à l'application...")
add_utterances(app_id, app_version)
print ()

Ajout d'exemple à l'application...
1 Exemple d’énoncé ajouté.



###  <font size=4 color="#3498DB "> 5. Apprentissage de l’application <a name="1"></a> </font>

In [96]:
def train_app(app_id, app_version):
    response = client.train.train_version(app_id, app_version)
    waiting = True
    while waiting:
        info = client.train.get_status(app_id, app_version)

        # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are done.
        waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, info))
        if waiting:
            print ("Attendez 10 secondes pour que l'apprentissage se termine...")
            time.sleep(10)

In [35]:
print ("Apprentissage de l’application ...")
train_app(app_id, app_version)
print ()

Apprentissage de l’application ...
Attendez 10 secondes pour que l'apprentissage se termine...



In [ ]:
def get_params(endpoint, app_version, prediction_key, app_id):
    """Renvoie les paramètres de l'app LUIS"""
    
    # On envoie la requête permettant d'exporter le modèle au format json
    response = requests.get(
        url=f"{endpoint}luis/authoring/v3.0-preview/apps/{app_id}/versions/{app_version}/export",
        params={
            "format": "json"
        },
        headers={
            "Ocp-Apim-Subscription-Key": prediction_key,
        }
    )
    
    # On vérifie la réponse
    check_response_ok_or_raise_for_status(response)
    
    # On renvoie les paramètres
    return response.json()

In [83]:
param = get_params(authoring_endpoint,"1",authoring_key, app_id)

In [84]:
JSON_PATH_LUIS = "/Users/kaju/Documents/FORMACAO IA OPENCLASSROOMS/PROJET 10 - DEVELOPPEZ UN CHATBOT POUR RESERVER DES VACANCES/P10_ARNONI_KAITO/ARNONI_KAITO_2_LUIS_042022/"
    

file_path = os.path.join(JSON_PATH_LUIS, "param.json")
with open(file_path, "w") as f:
    json.dump(param, f)

###  <font size=4 color="#3498DB "> 6. Publier l'application dans l’emplacement de production <a name="1"></a> </font>


In [102]:
def publish_app(app_id, app_version):
    client.apps.update_settings(app_id, is_public=True)
    responseEndpointInfo = client.apps.publish(app_id, app_version, is_staging=False)
    print("Point de terminaison: " + responseEndpointInfo.endpoint_url)


In [45]:
print ("On publie l'application...")
publish_app(app_id, app_version)

On publie l'application...
Point de terminaison: https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/4151fd40-18a7-4701-8d67-bf7ec96a511a


###  <font size=4 color="#3498DB "> 7. Obtenir une prédiction  <a name="1"></a> </font>

In [105]:
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient

prediction_key = os.getenv('AUTHORING_KEY')

prediction_endpoint = os.getenv('PREDICTIONENDPOINT')

runtimeCredentials = CognitiveServicesCredentials(prediction_key)
clientRuntime = LUISRuntimeClient(endpoint=prediction_endpoint, credentials=runtimeCredentials)

In [23]:
def predict(app_id, slot_name):

    request = { "query" : ["book flight from London to Paris on May 22, 2021 returning june 21, 2021 for $300"]}

    # Note be sure to specify, using the slot_name parameter, whether your application is in staging or production.
    response = clientRuntime.prediction.get_slot_prediction(app_id=app_id, slot_name=slot_name, prediction_request=request)

    print("Top intent: {}".format(response.prediction.top_intent))
    #print("Sentiment: {}".format (response.prediction.sentiment))
    print("Intents: ")

    for intent in response.prediction.intents:
        print("\t{}".format (json.dumps (intent)))
    print("Entities: {}".format (response.prediction.entities))
    return response.as_dict()

In [24]:
print ("Prédiction:")
pred = predict(app_id, "Production")

Prédiction:
Top intent: book_flight
Intents: 
	"book_flight"
Entities: {'from_city': ['London'], 'to_city': ['Paris'], 'from_dt': ['May 22, 2021'], 'to_dt': ['june 21, 2021'], 'budget': ['$300']}


Nous avons verifié que l'application est opérationelle. Maintenant nous allons suprimer cette prémiére version et créer une nouvelle avec tous les exemple d’énoncé à une intention du jeu d'entrainnement.

###  <font size=4 color="#3498DB "> 8. Supprimer la version <a name="1"></a> </font>

In [85]:
def delete(endpoint, app_version, app_key, app_id):
    
    # On crée le client avec les informations d'authentification
    client = LUISAuthoringClient(endpoint, prediction_key)

    # On delete la version de l'app
    client.versions.delete(app_id, app_version)

In [ ]:
delete(authoring_endpoint,"1",authoring_key, app_id)

###  <font size=4 color="#3498DB "> 9. Créer une nouvelle version <a name="1"></a> </font>

In [87]:
def new_version(authoring_endpoint, app_id, authoring_key, app_version, app_params: dict, app_utterances: list=[]):
    """Crée un nouvelle version de l'application"""
    
    # On ajoute les utterances aux paramètres de l'application
    app_params_tmp = app_params.copy()
    app_params_tmp["utterances"] += app_utterances
    
    # On envoie la requête permettant de créer la nouvelle version
    response = requests.post(
        url=f"{authoring_endpoint}luis/authoring/v3.0-preview/apps/{app_id}/versions/import",
        params={
            "versionId": app_version,
        },
        headers={
            "Ocp-Apim-Subscription-Key": authoring_key,
        },
        json=app_params_tmp
    )
    
    # On vérifie la réponse
    check_response_ok_or_raise_for_status(response)


In [88]:
file_path = os.path.join(JSON_PATH_LUIS, "param.json")
with open(file_path, "r") as f:
    app_params = json.load(f)

In [91]:
file_path = os.path.join(JSON_PATH_LUIS, "utterances_train.json")
with open(file_path, "r") as f:
    utterances_train = json.load(f)

In [94]:
app_version = "final"
new_version(authoring_endpoint, app_id, authoring_key, app_version, app_params, utterances_train)

In [99]:
print ("Apprentissage de l’application ...")
train_app(app_id, app_version)
print ()

Apprentissage de l’application ...
Attendez 10 secondes pour que l'apprentissage se termine...



In [100]:
param = get_params(authoring_endpoint,app_version,authoring_key, app_id)

file_path = os.path.join(JSON_PATH_LUIS, "param.json")
with open(file_path, "w") as f:
    json.dump(param, f)

**Publier l'application dans l’emplacement de production** 

In [103]:
print ("On publie l'application...")
publish_app(app_id, app_version)

On publie l'application...
Point de terminaison: https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/4151fd40-18a7-4701-8d67-bf7ec96a511a


**Evaluation du nouveau modèle**

In [106]:
def check_response_ok_or_raise_for_status(response):
    """Vérifie que la réponse est ok ou génère une erreur"""
    
    # On génère une exception en cas d'erreur
    if not response.ok:
        print(response.content)
        response.raise_for_status()

def evaluate(endpoint, app_id, prediction_key, utterances: list, check_status_period: int=10):
    """Evaluation de l'application LUIS sur un jeu de test"""
    
    # On envoie la requête permettant de lancer l'évaluation
    response = requests.post(
        url=f"{endpoint}luis/v3.0-preview/apps/{app_id}/slots/production/evaluations",
        headers={
            "Ocp-Apim-Subscription-Key": prediction_key,
        },
        json=utterances
    )
    
    # On vérifie la réponse
    check_response_ok_or_raise_for_status(response)
    
    # On récupère l'id de l'opération
    operation_id = response.json()["operationId"]
    
    waiting = True
    while waiting:
        # On check le status
        response = requests.get(
            url=f"{endpoint}luis/v3.0-preview/apps/{app_id}/slots/production/evaluations/{operation_id}/status",
            headers={
                "Ocp-Apim-Subscription-Key": prediction_key,
            }
        )
        
        # On vérifie s'il y a une erreur
        if response.json()["status"] == "failed":
            raise ValueError(response.content)
        
        waiting = response.json()["status"] in ["notstarted", "running"]
        
        if waiting:
            time.sleep(check_status_period)
        
    # On récupère les résultats de l'évaluation
    response = requests.get(
        url=f"{endpoint}luis/v3.0-preview/apps/{app_id}/slots/production/evaluations/{operation_id}/result",
        headers={
            "Ocp-Apim-Subscription-Key": prediction_key,
        }
    )
    
    # On récupère les résultats
    res = response.json()
    
    # On met en forme les résultats dans un DataFrame
    res = pd.DataFrame(res["intentModelsStats"] + res["entityModelsStats"])
    res.iloc[:, -3:] = res.iloc[:, -3:].astype(float)
    res.columns = [
        "model_name",
        "model_type",
        f"precision",
        f"recall",
        f"f_score",
    ]
    
    return res

In [107]:
file_path = os.path.join(JSON_PATH_LUIS, "utterances_test.json")
with open(file_path, "r") as f:
    utterances_test = json.load(f)

In [113]:
results = evaluate(prediction_endpoint, app_id, prediction_key, utterances_test)

In [114]:
results

,model_name,model_type,precision,recall,f_score
0,book_flight,Intent Classifier,0.93,0.97,0.95
1,None,Intent Classifier,0.90,0.83,0.86
2,budget,Entity Extractor,0.85,1.00,0.92
3,from_city,Entity Extractor,0.81,0.97,0.88
4,from_dt,Entity Extractor,0.86,1.00,0.92
5,to_city,Entity Extractor,0.91,0.95,0.93
6,to_dt,Entity Extractor,0.87,1.00,0.93


In [151]:
print(prediction_endpoint)

https://p10luis.cognitiveservices.azure.com/
